--- Day 18: Operation Order ---

As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their math homework.

Unfortunately, it seems like this "math" follows different rules than you remember.

The homework (your puzzle input) consists of a series of expressions that consist of addition (+), multiplication (*), and parentheses ((...)). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, the operators have the same precedence, and are evaluated left-to-right regardless of the order in which they appear.

For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are as follows:

```
1 + 2 * 3 + 4 * 5 + 6
  3   * 3 + 4 * 5 + 6
      9   + 4 * 5 + 6
         13   * 5 + 6
             65   + 6
                 71
```

Parentheses can override this order; for example, here is what happens if parentheses are added to form 1 + (2 * 3) + (4 * (5 + 6)):

```
1 + (2 * 3) + (4 * (5 + 6))
1 +    6    + (4 * (5 + 6))
     7      + (4 * (5 + 6))
     7      + (4 *   11   )
     7      +     44
            51
```

Here are a few more examples:

```
    2 * 3 + (4 * 5) becomes 26.
    5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 437.
    5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 12240.
    ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 13632.
```

Before you can help with the homework, you need to understand it yourself. Evaluate the expression on each line of the homework; what is the sum of the resulting values?


In [1]:
#!fsharp
let testInput = """1 + (2 * 3) + (4 * (5 + 6))
2 * 3 + (4 * 5)
5 + (8 * 3 + 9 + 3 * 4 * 3)
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2"""

let taskInput = System.IO.File.ReadAllText "task.txt"

In [1]:
#!fsharp
type Token = T_PLUS 
            | T_MUL
            | T_LPAREN
            | T_RPAREN
            | T_NUMBER of int64
            | T_EOF

let tokenizer (input:string) =
    seq {
        let trimmed = input.Replace(" ", "")
        let mutable pos = 0
        while pos < trimmed.Length do
            match trimmed.[pos] with
            | '+' ->
                pos <- pos + 1
                yield T_PLUS
            | '*' ->
                pos <- pos + 1
                yield T_MUL
            | '(' ->
                pos <- pos + 1
                yield T_LPAREN
            | ')' ->
                pos <- pos + 1
                yield T_RPAREN
            | ch when ch >= '0' && ch <= '9' ->
                let mutable value = 0L
                while pos < trimmed.Length && trimmed.[pos] >= '0' && trimmed.[pos] <='9' do
                    value <- value * 10L + (int64 trimmed.[pos] - int64 '0')
                    pos <- pos + 1
                yield T_NUMBER value
            | _ -> failwith ("Unexpected character " + trimmed.[..pos-1] + " ^^" + trimmed.[pos..])
        yield T_EOF
    }

tokenizer "((25 + 4 * 9) * (6 + 9 * 8) + 6)"


index,type,,Item
0,FSI_0009+Token,T_LPAREN,
1,FSI_0009+Token,T_LPAREN,
2,FSI_0009+Token+T_NUMBER,,25
3,FSI_0009+Token,T_PLUS,
4,FSI_0009+Token+T_NUMBER,,4
5,FSI_0009+Token,T_MUL,
6,FSI_0009+Token+T_NUMBER,,9
7,FSI_0009+Token,T_RPAREN,
8,FSI_0009+Token,T_MUL,
9,FSI_0009+Token,T_LPAREN,


In [1]:
#!fsharp
let evaluate (input: string) =
    let data = input |> tokenizer |> Array.ofSeq

    let rec eval start =
        let (result, pos) = evalOp start
        if data.[pos] <> T_EOF then
            failwithf "Expected end of expression, got %A" data.[pos]
        else
            (result, pos)
    and
        evalOp start =
            let (left, pos) = evalFactor start
            evalOp2 left pos
    and
        evalOp2 left start =
        match data.[start] with
        | T_PLUS ->
            let (right, pos) = evalFactor (start+1)
            evalOp2 (left+right) pos
        | T_MUL ->
            let (right, pos) = evalFactor (start+1)
            evalOp2 (left*right) pos
        | _ ->
            (left, start)
    and
        evalFactor start =
        match data.[start] with
        | T_NUMBER value ->
            (value, start+1)
        | T_LPAREN ->
            evalBracket start
        | token -> failwithf "Expected ( or number, got %A" token
    and evalBracket start =
        match data.[start] with
        | T_LPAREN ->
            let (res,pos) = evalOp (start+1)
            if data.[pos] <> T_RPAREN then
                failwithf "Expected ), got %A as %i" data.[pos] pos
            else
                (res, pos+1)
        | _ -> evalOp start
        
    eval 0 |> fst

evaluate "5+1+(5*6)"

36

In [1]:
#!fsharp
let evaluateAll (input: string) =
    input.Replace("\r", "").Split("\n")
    |> Seq.map evaluate

evaluateAll testInput

index,value
0,26
1,437
2,12240
3,13632


In [1]:
#!fsharp
taskInput |> evaluateAll |> Seq.sum

25190263477788

--- Part Two ---

You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: advanced math.

Now, addition and multiplication have different precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated before multiplication.

For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are now as follows:

```
1 + 2 * 3 + 4 * 5 + 6
  3   * 3 + 4 * 5 + 6
  3   *   7   * 5 + 6
  3   *   7   *  11
     21       *  11
         231
```

Here are the other examples from above:

```
    1 + (2 * 3) + (4 * (5 + 6)) still becomes 51.
    2 * 3 + (4 * 5) becomes 46.
    5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 1445.
    5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 669060.
    ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 23340.
```

What do you get if you add up the results of evaluating the homework problems using these new rules?


In [1]:
#!fsharp
let evaluate2 (input: string) =
    let data = input |> tokenizer |> Array.ofSeq

    let rec eval start =
        let (result, pos) = evalMul start
        if data.[pos] <> T_EOF then
            failwithf "Expected end of expression, got %A at %i" data.[pos] pos
        else
            (result, pos)
    and
        evalMul start =
            let mutable (left, pos) = evalAdd start
            while data.[pos] = T_MUL do
                let (right, pos2) = evalAdd (pos+1)
                left <- left * right
                pos <- pos2

            (left, pos)
    and
        evalAdd start =
            let mutable (left, pos) = evalFactor start
            while data.[pos] = T_PLUS do                
                let (right, pos2) = evalFactor (pos+1)
                left <- left + right
                pos <- pos2

            (left, pos)
    and
        evalFactor start =
            match data.[start] with
            | T_NUMBER value ->
                (value, start+1)
            | T_LPAREN ->
                evalBracket (start+1)
            | token -> failwithf "Expected ( or number, got %A at %i" token start
    and evalBracket start =
            let (res,pos) = evalMul start
            if data.[pos] <> T_RPAREN then
                failwithf "Expected ), got %A at %i" data.[pos] pos
            else
                (res, pos+1)
        
    eval 0 |> fst

evaluate2 "8*1+5"

48

In [1]:
#!fsharp
let evaluateAll2 (input: string) =
    input.Replace("\r", "").Split("\n")
    |> Seq.map evaluate2

evaluateAll2 testInput

index,value
0,51
1,46
2,1445
3,669060
4,23340


In [1]:
#!fsharp
taskInput |> evaluateAll2 |> Seq.sum

297139939002972